In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import math

In [ ]:
def test(camera_positions, scenes, QPs, synthesizers, numGroups, pose_traces, numFrame):
    df = pd.DataFrame()
    df_tmp = pd.read_csv(f'./results/all_nG1.csv')
    df = df.append(df_tmp)

    
    
    
    for scene in scenes:
        scene = 'office'
        print(scene)
        df_tmp = df
        df_tmp = df_tmp.loc[(df[f'synthesizer'] == 'AdditiveSynthesizer') & (df["camera_position"] != 'sphere_6x6')]
        df_tmp = df_tmp.loc[(df["scene"] == scene) & (df["QP"] == 20)]
        
        # # PSNR
        # ax = sns.barplot(x='camera_position', y=f'psnr_mean', hue='pose_traces', data=df_tmp)
        # ax.set(xlabel="Camera placement", ylabel=f"PSNR(db)")
        # plt.legend(title="")
        # # plt.ylim((40,60))   
        # # plt.savefig(f'./img/synthesizer/psnr_camera_position_synthesizer_in_rec20_{scene}.eps', dpi=300)
        # # plt.savefig(f'./img/synthesizer/psnr_camera_position_synthesizer_in_rec20_{scene}.png', dpi=300)
        # plt.show()

        # # SSIM
        # ax = sns.barplot(x='camera_position', y=f'ssim_mean', hue='pose_traces', data=df_tmp)
        # ax.set(xlabel="Camera placement", ylabel=f"SSIM")
        # plt.legend(title="")
        # # plt.ylim((0,1.0))   
        # # plt.savefig(f'./img/synthesizer/psnr_camera_position_synthesizer_in_rec20_{scene}.eps', dpi=300)
        # # plt.savefig(f'./img/synthesizer/psnr_camera_position_synthesizer_in_rec20_{scene}.png', dpi=300)
        # plt.show()

        ax = sns.barplot(x='camera_position', y=f'vmaf_mean', hue='pose_traces', data=df_tmp)
        ax.set(xlabel="Camera placement", ylabel=f"VMAF")
        plt.legend(title="")
        # plt.ylim((0,100))   
        # plt.savefig(f'./img/synthesizer/psnr_camera_position_synthesizer_in_rec20_{scene}.eps', dpi=300)
        # plt.savefig(f'./img/synthesizer/psnr_camera_position_synthesizer_in_rec20_{scene}.png', dpi=300)
        plt.show()

In [ ]:
def main():
    # setup paras
    # camera_positions = np.array(["6x6", "9x4", "12x3", "18x2"])
    # scenes = np.array(['ArchVizInterior', 'LightroomInteriorDayLight', 'office', 'RealisticRendering', 'XoioBerlinFlat'])
    # QPs = np.array([20, 36, 44, 48, 50])
    # synthesizers = np.array(["AdditiveSynthesizer", "ViewWeightingSynthesizer"])
    # numGroups = np.array([1])
    # pose_traces = np.array([f'pose{i}' for i in range(10)])
    # numFrame = 1

    # 1
    camera_positions = np.array(["6x6", "9x4", "12x3", "18x2"])
    scenes = np.array(['ArchVizInterior', 'LightroomInteriorDayLight', 'office', 'RealisticRendering', 'XoioBerlinFlat'])
    QPs = np.array([20, 36, 44, 48, 50])
    synthesizers = np.array(["AdditiveSynthesizer", "ViewWeightingSynthesizer"])
    numGroups = np.array([1, 4])
    pose_traces = np.array([f'pose{i}' for i in range(10)])
    numFrame = 1


In [ ]:
if __name__ == '__main__':
    main()